In [ ]:
import os
import numpy as np
import mne
import imageio
import h5py
#import scipy.fftpack
import matplotlib
%matplotlib inline
import pywt
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
#from scipy import signal
from matplotlib.colors import ListedColormap
import time
import seaborn as sns
#import scipy.io as sio
#from scipy.integrate import simps
import pandas as pd
#from scipy import fft
import matplotlib.mlab as mlab
import sys
sys.path.append('T:\EL_experiment\Codes\CCEP_human\Python_Analysis/py_functions')
import analys_func
from scipy.stats import norm
import LL_funcs as LLf
from scipy.stats import norm
from tkinter import filedialog
from tkinter import *
import ntpath
root = Tk()
root.withdraw()
import math
import scipy
from scipy import signal
import pylab
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
import platform
from glob import glob
from scipy.io import savemat

import BM_func as BMf
import basic_func as bf
from matplotlib.patches import Rectangle
import tqdm
import similarity_funcs as sf

import freq_funcs as ff
subj            = "EL011"
cwd             = os.getcwd()

path_patient_analysis = 'y:\\eLab\EvM\Projects\EL_experiment\Analysis\Patients\\' + subj
path_gen = os.path.join('y:\\eLab\Patients\\' + subj)
if not os.path.exists(path_gen):
    path_gen = 'T:\\EL_experiment\\Patients\\' + subj
path_patient = path_gen + '\Data\EL_experiment'  # os.path.dirname(os.path.dirname(cwd))+'/Patients/'+subj
path_infos = os.path.join(path_patient, 'infos')
if not os.path.exists(path_infos):
    path_infos = path_gen + '\\infos'

color_elab      = ['#594157', "#F1BF98","#8FB996"]
t_label         = ['Baseline', 'Flumazenil', 'Benzodiazepin']  # ['Baseline', 'Flumazenil', 'Benzodiazepin']
lbls = pd.read_excel(os.path.join(path_infos, subj + "_labels.xlsx"), header=0, sheet_name='BP')
labels_all      = lbls.label.values
labels_clinic   = lbls.Clinic.values
labels_region   = lbls.Region.values
labels_L        = lbls.Hemisphere.values
nodes           = pd.DataFrame({'ID': labels_all, 'Region': labels_region, 'H': labels_L})
coord_all       = np.array([lbls.x.values,lbls.y.values, lbls.z.values ]).T
bad_region = np.where((labels_region=='WM')|(labels_region=='OUT')|(labels_region=='Putamen'))[0]
#if subj == 'EL006':
 #     labels_all      = lbls.Clinic.values
badchans  = pd.read_csv(path_patient_analysis + '/BrainMapping/data/badchan.csv')
bad_chans = np.unique(np.array(np.where(badchans.values[:,1:]==1))[0,:])

bad_stims = np.where(labels_region=='OUT')[0]

##all 
cond_vals   = np.arange(4)
cond_labels = ['BM', 'BL', 'Fuma', 'BZD']
cond_colors = ['#494159','#594157', "#F1BF98","#8FB996"]

In [ ]:
Fs             = 500
dur            = np.zeros((1,2), dtype=np.int32)
t0 = 1
dur[0,0] =  -t0
dur[0,1] =  3

#dur[0,:]       = np.int32(np.sum(abs(dur)))
x_ax           = np.arange(dur[0,0],dur[0,1],(1/Fs))
color_elab      = np.zeros((3,3))
color_elab[0,:] = np.array([31, 78, 121])/255
color_elab[1,:] = np.array([189, 215, 238])/255
color_elab[2,:] = np.array([0.256, 0.574, 0.431])

In [ ]:
folder = 'BrainMapping'
cond_folder = 'CR'

In [ ]:
EEG_CR_file = path_patient_analysis + '\\' + folder + '\\' + cond_folder + '\\data\\EEG_CR.npy'
EEG_resp = np.load(EEG_CR_file)
stimlist = pd.read_csv(path_patient_analysis + '\\' + folder + '\\' + cond_folder + '\\data\\Stimlist_CR.csv')


In [ ]:
#file_MN1 = path_patient_analysis + '\\' + folder + '\\data\\M_N1.npy'
file_con = path_patient_analysis + '\\' + folder + '\\' + cond_folder + '\\data\\con_trial_all.csv'
con_trial    = pd.read_csv(file_con)
#M_N1peaks = np.load(file_MN1)

In [ ]:
days = np.unique(stimlist.date)
# get stimulation channels directly from stimlist
StimChanSM   = np.unique(stimlist.ChanP)
ChanN        = np.zeros((len(StimChanSM),))
StimChans    = []#np.zeros((len(stim_chan)))
StimChansC   = []#np.zeros((len(stim_chan)))
StimChanIx   = []#np.zeros((len(stim_chan)))
for i in range(len(StimChanSM)):
    ChanN[i] =  np.median(stimlist[stimlist.ChanP==StimChanSM[i]].ChanN)
    #StimChans.append(labels_SM[(np.array(labels.chan_num.values)==stim_chan[i,0])][0])
    StimChans.append(labels_all[(np.array(lbls.ChanP_SM.values)==StimChanSM[i])&(np.array(lbls.ChanN_SM.values)==ChanN[i])][0])
    StimChansC.append(labels_clinic[(np.array(lbls.ChanP_SM.values)==StimChanSM[i])&(np.array(lbls.ChanN_SM.values)==ChanN[i])][0])
    StimChanIx.append(lbls[(np.array(lbls.ChanP_SM.values)==StimChanSM[i])&(np.array(lbls.ChanN_SM.values)==ChanN[i])]['Num'].values[0]-1)
    # bad channels

## Mean resps

## LL and Significant responses 

In [ ]:
stimlist.insert(0,'h',stimlist.hour)

In [ ]:
exp    = 1
w_r    = 0.1
w_LL   = 0.25
t_0    = 1 # time of stimulation in data
t_0s   = 0.5 # time of surr stim
t_Bl   = 0.5

## calcualte mean CCEp and then take LL 
data_CCEP                = np.zeros((1,10))
w                        = 0.25
stim_spec                = stimlist[(stimlist.IPI_ms ==0)]#&(stimlist.noise ==0)]#&(stimlist.noise ==0)
stimNum                  = stim_spec.StimNum.values#[:,0]
resps                    = ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs)
ChanP1                   = bf.SM2IX(stim_spec.ChanP.values,StimChanSM,np.array(StimChanIx))
LL_all          = LL_funcs.get_LL_both(data=resps, Fs=Fs, IPI=np.zeros((len(stimNum),1)), t_0=t_0, win=w_LL)
LL_r            = LL_funcs.get_LL_ratio(data=resps, Fs=Fs, IPI=np.zeros((len(stimNum),1)),t_bl=t_0-0.2, t_stim=t_0, win=w_r)
#LL_CCEP[sc, :, h, 0] = 
# remove Resp if rec channel was just stimulating before
stim_spec0                = stimlist[(stimlist.StimNum.isin((stim_spec.StimNum.values-1)[1:]))]
ChanP0                    = np.zeros((len(stimNum),))
ChanP0[1:]                = bf.SM2IX(stim_spec0.ChanP.values,StimChanSM,np.array(StimChanIx))

for c in range(LL_all.shape[0]):
    val         = np.zeros((LL_all.shape[1], 10))
    val[:, 0]   = c                                         # response channel
    val[:, 1]   = bf.SM2IX(stim_spec.ChanP.values,StimChanSM,np.array(StimChanIx)) #stim Chan, in all labels
    val[:, 2]   = LL_all[c,:,1] ## LL absolute
    val[:, 3]   = stimNum
    val[:, 4]   = LL_r[c,:,1]
    val[:, 6]   = stim_spec.h.values
    val[:, 7]   = stim_spec.date.values
    #val[:, 8]   = np.max(abs(resps[c,:,np.int64(1.015*Fs):np.int64(1.2*Fs)]),1)
    #val[:, 9]   = np.max(abs(resps[c,:,np.int64(1.015*Fs):np.int64(1.2*Fs)]),1)/np.std(resps[c,:,0:np.int64(0.9*Fs)],1)
    # remove value if response channel is or jsut was aprt of stimulation
    val[np.where(bf.check_inStimChan(c, ChanP0, labels_clinic)==1),2] = np.nan
    val[np.where(bf.check_inStimChan(c, ChanP0, labels_clinic)==1),4] = np.nan
    #val[np.where(bf.check_inStimChan(c, ChanP0, labels_clinic)==1),8] = np.nan
    #val[np.where(bf.check_inStimChan(c, ChanP0, labels_clinic)==1),9] = np.nan
    val[np.where(bf.check_inStimChan(c, ChanP1, labels_clinic)==1),2] = np.nan
    val[np.where(bf.check_inStimChan(c, ChanP1, labels_clinic)==1),4] = np.nan
    #val[np.where(bf.check_inStimChan(c, ChanP1, labels_clinic)==1),9] = np.nan
    #val[np.where(bf.check_inStimChan(c, ChanP1, labels_clinic)==1),8] = np.nan

    #ix         = np.where(np.max(abs(resps[c,:,np.int64(0.95*Fs):np.int64(1.01*Fs)]),1)>400)
    pks         = np.max(abs(resps[c,:,np.int64(0.95*Fs):np.int64(1.25*Fs)]),1)
    pks_loc     = np.argmax(abs(resps[c,:,np.int64(0.95*Fs):np.int64(1.1*Fs)]),1)+np.int64(0.95*Fs)
    #ix         = np.where(np.max(abs(resps[c,:,np.int64(0.95*Fs):np.int64(1.01*Fs)]),1)>400)
    ix = np.where((pks>100)&(pks_loc>np.int64(0.955*Fs))&(pks_loc<np.int64(1.012*Fs)))
    val[ix, 2] = np.nan
    val[ix, 4] = np.nan
    #val[ix, 8] = np.nan
    #val[ix, 9] = np.nan
    data_CCEP    = np.concatenate((data_CCEP, val), axis=0)

data_CCEP = data_CCEP[1:-1, :] # remove first row (dummy row)

#LL_CCEP = pd.DataFrame(
#    {"Chan": data_CCEP[:, 0], "Stim": data_CCEP[:, 1], "LL": data_CCEP[:, 2],"d": data_CCEP[:, 5],"rLL": data_CCEP[:, 4],"zLL": data_CCEP[:, 4],"mx": data_CCEP[:, 8],"mx/std": data_CCEP[:, 9],"Day": data_CCEP[:, 7],"Hour": data_CCEP[:, 6],"Num": data_CCEP[:, 3]})
#
LL_CCEP = pd.DataFrame(
    {"Chan": data_CCEP[:, 0], "Stim": data_CCEP[:, 1], "LL": data_CCEP[:, 2],"d": data_CCEP[:, 5],"rLL": data_CCEP[:, 4],"zLL": data_CCEP[:, 4],"Day": data_CCEP[:, 7],"Hour": data_CCEP[:, 6],"Num": data_CCEP[:, 3]})

LL_CCEP.loc[LL_CCEP['Chan'].isin(bad_chans), 'LL'] = np.nan

##Z-score absolute
LL_BL_z   = np.zeros((len(labels_all),2))
LL_all_BL       = LL_funcs.get_LL_both(data=resps, Fs=Fs, IPI=np.zeros((len(stimNum),1)), t_0=t_Bl, win=w_LL)
LL_BL_z[:,0] = np.nanmean(LL_all_BL[:,:,1],1)
LL_BL_z[:,1] = np.nanstd(LL_all_BL[:,:,1],1)
for rc in range(len(labels_all)):
    LL_CCEP.loc[(LL_CCEP.Chan ==rc), 'zLL'] = (LL_CCEP.loc[(LL_CCEP.Chan ==rc), 'LL']- LL_BL_z[rc,0])/LL_BL_z[rc,1]

## Resp based on absolute LL 99th        
LL_CCEP.insert(0,'RespA', 0)
thr = np.percentile(LL_all_BL[:,:,1], 99,1)
for rc in range(len(labels_all)):
    LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.LL>thr[rc]), 'RespA'] = 1
LL_CCEP.loc[(LL_CCEP.LL==-2), 'RespA'] = 0  


## Resp based on absolute LL 99th        
LL_CCEP.insert(0,'RespR', 0)
LL_BL_ratio      = LL_funcs.get_LL_ratio(data=ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), Fs=Fs, IPI=np.zeros((len(stimNum),1)),t_bl=t_0s-0.2, t_stim=t_0s, win=w_r)
thr              = np.percentile(LL_BL_ratio[:,:,1], 99,1)
for rc in range(len(labels_all)):
    LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.rLL>thr[rc]), 'RespR'] = 1
LL_CCEP.loc[np.isnan(LL_CCEP.LL), 'RespR'] = 0  

#LL_CCEP.insert(0,'RespM', 0)
#LL_CCEP.loc[(LL_CCEP['mx/std']>4), 'RespM'] = 1                
# distance
for i in range(len(StimChans)):
    ChanP = StimChanSM[i]
    s   = np.where(labels_all == StimChans[i])[0][0]#i#np.int(StimChanNums[i]) 
    s   = np.int64(s)
    for c in np.unique(LL_CCEP.Chan):
        c   = np.int64(c)
        LL_CCEP.loc[(LL_CCEP.Stim == s)&(LL_CCEP.Chan == c), 'd'] = math.sqrt(((coord_all[s,0]-coord_all[c,0])**2)+((coord_all[s,1]-coord_all[c,1])**2)+((coord_all[s,2]-coord_all[c,2])**2))

In [ ]:
LL_CCEP

In [ ]:
LL_CCEP, M_resp = bf.sign_conncetion(LL_CCEP, EEG_resp, labels_all, dur)

In [ ]:
M_resp = np.zeros((len(labels_all), len(labels_all)))-1
for sc in tqdm.tqdm(range(len(labels_all))):
    for rc in range(len(labels_all)):
        M_resp[sc, rc] = np.mean(LL_CCEP.loc[(LL_CCEP.Stim==sc)&(LL_CCEP.Chan==rc), 'RespC'])


In [ ]:
M_resp[np.isnan(M_resp)] = -1

In [ ]:
import tqdm

In [ ]:
n   = 10 
w   = 0.1
M_resp = np.zeros((len(labels_all), len(labels_all)))
for sc in tqdm.tqdm(range(len(labels_all))):
    for rc in range(len(labels_all)):
        lists          = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)]
        lists          = lists[~np.isnan(lists.zLL.values)]
        stimNum_all    = lists.Num.values.astype('int')

        if len(stimNum_all)>0:
            #list_BL        = LL_CCEP[(LL_CCEP['d']>20)&(LL_CCEP['Chan']==rc)&~(LL_CCEP['Stim']==sc)&~np.isnan(LL_CCEP.zLL.values)]
            #stimNum_BL     = list_BL.Num.values.astype('int')
            resp           = ff.lp_filter(np.nanmean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
            #resp_BL        = np.zeros((n, np.sum(abs(dur))*Fs))
            #for k in range(n):
            #    resp_BL[k,:] = ff.lp_filter(np.nanmean(EEG_resp[rc,np.random.choice(stimNum_BL, len(stimNum_all)),:],0),45,Fs)
            #resp_BL = np.flip(resp_BL,1)
            #LL_BL     = get_LL_all(np.expand_dims(resp_BL, axis=0), Fs, w, 1, 0)
            LL_resp   = get_LL_all(np.expand_dims(np.expand_dims(resp, axis=0),0), Fs, w, 1, 0)
            #LL_BL     = LL_BL[0]
            LL_resp   = LL_resp[0,0]
            thr        = np.percentile(LL_BL[:,0:np.int64((1-w/2)*Fs)],99.9)
            LL_thr = LL_resp-thr
            
            #LL_BL = np.concatenate([LL_BL, np.expand_dims(LL_resp,0)])
            if np.max(LL_resp[np.int64(Fs*(1.01+w/2)):np.int64(Fs*(1.01+w))])>np.percentile(LL_BL[:,0:np.int64(2*Fs)],99.9):
                M_resp[sc,rc] = 1
                LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.Stim==sc), 'RespC'] = 1
            else:
                LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.Stim==sc), 'RespC'] = 0
        else:
            M_resp[sc,rc] = -1

In [ ]:
n   = 10 
w   = 0.1
M_resp = np.zeros((len(labels_all), len(labels_all)))-1
for sc in tqdm.tqdm(range(len(labels_all))):
    for rc in range(len(labels_all)):
        lists          = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)]
        lists          = lists[~np.isnan(lists.zLL.values)]
        stimNum_all    = lists.Num.values.astype('int')

        if len(stimNum_all)>0:
            list_BL        = LL_CCEP[(LL_CCEP['d']>20)&(LL_CCEP['Chan']==rc)&~(LL_CCEP['Stim']==sc)&~np.isnan(LL_CCEP.zLL.values)]
            stimNum_BL     = list_BL.Num.values.astype('int')
            resp_BL        = np.zeros((n, np.sum(abs(dur))*Fs))
            for k in range(n):
                resp_BL[k,:] = ff.lp_filter(np.nanmean(EEG_resp[rc,np.random.choice(stimNum_BL, len(stimNum_all)),:],0),45,Fs)
            LL_BL     = get_LL_all(np.expand_dims(resp_BL, axis=0), Fs, w, 1, 0)
            LL_BL     = LL_BL[0]
    
            resp       = ff.lp_filter(np.nanmean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
            LL_resp    = get_LL_all(np.expand_dims(np.expand_dims(resp, axis=0),0), Fs, w, 1, 0)
            LL_resp    = LL_resp[0,0]
            
            LL_meregd = np.concatenate([LL_BL, np.expand_dims(LL_resp,0)])
            thr        = np.percentile(LL_meregd[0:np.int64((1-w/2-0.05)*Fs)],99)
            if  np.mean(LL_resp[np.int64((1.01+w/2)*Fs):np.int64((1+w)*Fs)]>thr)>0.9:
                M_resp[sc,rc] = 1
                LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.Stim==sc), 'RespC'] = 1
            else:
                M_resp[sc,rc] = 0
                LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.Stim==sc), 'RespC'] = 0


In [ ]:
LL_CCEP.to_csv(path_patient + '/Analysis/BrainMapping/LL/CCEP_'+str(exp)+'_'+str(w)+'s.csv', index=False,
              header=True)  # scat_plot = scat_plot.fillna(method='ffill')
print('Data saved')

In [ ]:
regions       = np.unique(labels_region)
color_regions = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6',"#8FB996"]

v= np.sort(labels_region)
region_border = np.where(np.roll(v,1)!=v)[0]
region_border = np.concatenate([region_border, [len(labels_all)]])
region_num = np.diff(region_border)
labels_sort     = labels_all[np.argsort(labels_region)]
labels_region_sort = np.sort(labels_region)
StimChanIx_sort = np.argsort(labels_region)

In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
M_resp_region = M_resp[np.argsort(labels_region),:]
M_resp_region = M_resp_region[:,np.argsort(labels_region)]
cmap     = ListedColormap(['white', 'black', 'red'])
fig      = pylab.figure(figsize=(20,20))
axmatrix = fig.add_axes([0.15,0.15,0.9,0.9])
im       = axmatrix.matshow(M_resp_region, aspect='auto', origin='lower',cmap= cmap)
for i in range(len(region_border)-1):
    axmatrix.add_patch(Rectangle((-1.5, region_border[i]-0.5), 1, region_num[i], color = color_regions[np.where(regions==labels_region_sort[region_border[i]])[0][0]]))
    axmatrix.add_patch(Rectangle((region_border[i]-0.5, len(labels_all)-0.5),  region_num[i],1, color = color_regions[np.where(regions==labels_region_sort[region_border[i]])[0][0]]))
    t = plt.text(-20, (region_border[i]+region_num[i]/2)-1, np.unique(v)[i], fontsize=15)
    t.set_bbox(dict(facecolor=color_regions[np.where(regions==labels_region_sort[region_border[i]])[0][0]], alpha=0.5))
plt.xlim([-1.5, len(labels_all)+1])
plt.ylim([0, len(labels_all)+0.5])
plt.xticks(range(len(labels_all)), labels_sort, rotation=90);
plt.yticks(range(len(labels_all)), labels_sort);
#filename    = path_patient + '/Analysis/BrainMapping/LL/figures/BM_plot/BM_general.png'
#plt.savefig(filename)
plt.show()

In [ ]:
data_test = LL_CCEP[(LL_CCEP.RespC==1)&(LL_CCEP.d>7.5)] 
for c in range(len(labels_all)):
    data_test.loc[(data_test.Chan ==c), "Recs"]   = labels_all[c]
    data_test.loc[(data_test.Stim ==c), "Stim Region"]   = labels_region[c]
    data_test.loc[(data_test.Chan ==c), "Resp Region"]   = labels_region[c]
    data_test.loc[(data_test.Stim ==c), "Stims"]  = labels_all[c]

In [ ]:
data_asym     = np.zeros((1,6))
lists         = LL_CCEP[(LL_CCEP['RespC']==1)]
lists         = lists[~np.isnan(lists.LL.values)]
for s in range(len(labels_all)):
    for c in range(len(labels_all)):
        list_1 = lists[(lists.Chan==c)&(lists.Stim==s)]
        if len(list_1)>0:
            val         = np.zeros((1, 6))
            val[0, 0]   = c                                         # response channel
            val[0, 1]   = s                                         # response channel
            val[0, 4]   = np.nanmean(list_1.d)
            data_asym    = np.concatenate((data_asym, val), axis=0)
data_asym = data_asym[1:-1, :] # remove first row (dummy row)
data_A = pd.DataFrame(
    {"Chan": data_asym[:, 0], "Stim": data_asym[:, 1], "R": data_asym[:, 2], "Diff": data_asym[:, 2], "d": data_asym[:, 4]})

for c in range(len(labels_all)):
    data_A.loc[(data_A.Chan ==c), "Recs"]   = labels_all[c]
    data_A.loc[(data_A.Stim ==c), "Stim Region"]   = labels_region[c]
    data_A.loc[(data_A.Chan ==c), "Resp Region"]   = labels_region[c]
    data_A.loc[(data_A.Stim ==c), "Stims"]  = labels_all[c]
data_A.insert(5,'Change',0)
data_A=data_A.drop(data_A[data_A['Stim Region']=='OUT'].index)
data_A=data_A.drop(data_A[data_A['Resp Region']=='OUT'].index)

In [ ]:
#fig, ax = plt.figure(figsize=(13,13))

fig, ax     = plt.subplots(figsize=(15,12))
plt.title(subj+' - Connection decrease by Benzo')
k=0

data_test = data_A[(data_A.d>7.5)] 
for i in range(len(labels_all)):
    s = StimChanIx_sort[i]
    ax.add_patch(Rectangle((6.5, i-0.5), 1, 1, color = color_regions[np.where(regions==labels_region[s])[0][0]]))
    data_plot = data_test[data_test.Stim==s]
    for j in range(len(data_plot)):
        #plt.plot(data_plot.d.values[j], i,'o', markersize=10, c= color_regions[np.where(regions==data_plot['Resp Region'].values[j])[0][0]])
        plt.scatter(data_plot.d.values[j], i, s=80, c=color_regions[np.where(regions==data_plot['Resp Region'].values[j])[0][0]], alpha=0.8, edgecolors=[0,0,0])
    if any (region_border == i):
        num= region_num[np.where(region_border==i)[0][0]]
        t = plt.text(-15, (i+num/2)-1, v[i], fontsize=15)
        #t = plt.text(0.5, 0.5, 'text', transform=ax.transAxes, fontsize=30)
        t.set_bbox(dict(facecolor=color_regions[np.where(regions==labels_region[s])[0][0]], alpha=0.5, edgecolor=color_regions[np.where(regions==labels_region[s])[0][0]]))


plt.yticks(np.arange(len(labels_all)),labels_sort, fontsize=7)
plt.xlim([6.5, 105])
plt.ylim([-0.5, len(labels_all)-0.5])
plt.xlabel('euclidean distance [mm]')
plt.show()

In [ ]:
sig_resp = pd.DataFrame(M_resp)
sig_resp.to_csv(path_patient + '/Analysis/all/sign_chan.csv', index=False,
              header=True) 

In [ ]:
LL_CCEP =pd.read_csv(path_patient + '/Analysis/BrainMapping/LL/CCEP_'+str(exp)+'_'+str(w)+'s.csv')

In [ ]:
np.max(M_resp)

In [ ]:
fig      = pylab.figure(figsize=(20,20))
axmatrix = fig.add_axes([0.15,0.15,0.9,0.9])
cmap     = ListedColormap(['white', 'black', 'silver'])
im = axmatrix.matshow(M_resp, aspect='auto', origin='lower',cmap= cmap)

plt.xticks(range(len(labels_all)), labels_all, rotation=90);
plt.yticks(range(len(labels_all)), labels_all);

plt.show()

### example

In [ ]:
#w =0.1
#d = 15
#r = 1
#c = 1
#dat      = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP['RespC'] ==c)&(LL_CCEP['d'] >d)&(LL_CCEP['RespR'] ==r)]
#
#k        = np.random.randint(0, len(dat))
#stimNum  = np.int64(dat.Num.values[k])
#sc = np.int64(dat.Stim.values[k])
#rc = np.int64(dat.Chan.values[k])

j = 3

sc = 30
rc = 43
lists          = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)]
lists          = lists[~np.isnan(lists.zLL.values)]
stimNum_all    = lists.Num.values.astype('int')
n= 100
if len(stimNum_all)>0:
    resp           = ff.lp_filter(np.nanmean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
    fig   = plt.figure(figsize=(10,10) )
    plt.suptitle(labels_all[sc]+' -- '+labels_all[rc]+', Distance: '+str(np.round(lists.d.values[0],2))+'mm')
    gs    = fig.add_gridspec(2,1)  # GridSpec(4,1, height_ratios=[1,2,1,2])
    ax   = fig.add_subplot(gs[0,0])

    plt.xlim([-1, 1])
    
    plt.axvspan(0.01, w, alpha=0.2, color=[1,0,0])
    plt.axvspan(0.015+w/2,0.015+3*w/2, color=[1,0.5,0], alpha=0.1)
    for k in range(len(stimNum_all)):
        plt.plot(x_ax, ff.lp_filter(EEG_resp[rc,stimNum_all[k],:],45,Fs))
    plt.plot(x_ax,resp, c=[0,0,0], linewidth=3, label='mean resp, n:'+str(len(stimNum_all)))
    plt.title('Mean Responses')
    #plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
    plt.ylim([-np.min([np.max([150,np.max(abs(resp))*2]),800]),np.min([np.max([150,np.max(abs(resp))*2]),800])])
    #plt.ylim([-100,100])
    plt.ylabel('[uV/ms]')
    plt.axvline(0, c=[0,0,0])
    ax   = fig.add_subplot(gs[1,0], sharex=ax)

    
    LL_resp   = get_LL_all(np.expand_dims(np.expand_dims(resp, axis=0),0), Fs, w, 1, 0)
    LL_resp   = LL_resp[0,0]
    plt.axvline(0, c=[0,0,0])
    #plt.axvline(0+0.01+w/2, c=[0,0,0])
    #plt.axvspan(0.01+w/2, resp, alpha=0.8, color=color_elab[1])

   # for k in range(n):
   #     plt.plot(x_ax,LL_BL[k], c=color_elab[0],alpha=0.7, linewidth=3)
   #     #plt.plot(x_ax,LL_resp[k], c=[0,0,0], linewidth=3)
    plt.plot(x_ax,LL_resp, c=[0,0,0], linewidth=3)
    plt.plot(np.argmax(LL_resp[np.int64(Fs*(1.01+w/2)):np.int64(Fs*(1.01+1.5*w))])/Fs+0.01+w/2,np.max(LL_resp[np.int64(Fs*(1.01+w/2)):np.int64(Fs*(1.01+1.5*w))]), 'or')
    plt.title('LL of Mean Responses')
    #plt.axhline(np.percentile(LL_BL[:,0:np.int64(2*Fs)],99), c=[1,0,0], label ='99th percentile, entire 2s')#0:np.int64((1-w/2)*Fs)
    LL_meregd = np.concatenate([LL_BL, np.expand_dims(LL_resp,0)])
    #plt.axhline(np.percentile(LL_meregd[:,0:np.int64((2)*Fs)],99), c=[0.8,0,1], label ='99th percentile')#0:np.int64((1-w/2)*Fs)
    #plt.axhline(np.percentile(LL_resp[0:np.int64((1-w/2)*Fs)],99), c=[0.8,0,1], label ='99th percentile')#0:np.int64((1-w/2)*Fs)
    
    
    plt.ylim([0,np.max([2,LL_resp[np.int64(Fs*(1.01+w/2))]*1.1])])
    #plt.ylim([0,1.5])
    plt.ylabel('0.25s LL [uV/ms]')

    thr        = np.percentile(LL_resp[0:np.int64((1-w/2)*Fs)],99)
    plt.axhline(thr, c=[1,0,0], label ='99th percentile')#0:np.int64((1-w/2)*Fs)
    plt.axvline(0.01+w/2, c=[1,0,0], alpha=0.8)
    plt.axvline(0.01+w, c=[1,0.5,0], alpha=0.8)
    plt.legend()
    if  all(LL_resp[np.int64((1.015+w/2)*Fs):np.int64((1.015+1.5*w)*Fs)]>thr):
        print('yes')

In [ ]:
w = 0.1
d = 15

c = 1
dat      = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))]

k        = np.random.randint(0, len(dat))
stimNum  = np.int64(dat.Num.values[k])
sc = 36#np.int64(dat.Stim.values[k])
rc = 28#np.int64(dat.Chan.values[k])
n= 10
lists          = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)]
lists          = lists[~np.isnan(lists.zLL.values)]
stimNum_all    = lists.Num.values.astype('int')
if len(stimNum_all)>0:
    list_BL        = LL_CCEP[(LL_CCEP.RespR==0)&(LL_CCEP['d']>20)&(LL_CCEP['Chan']==rc)&~(LL_CCEP['Stim']==sc)&~np.isnan(LL_CCEP.zLL.values)]
    stimNum_BL     = list_BL.Num.values.astype('int')
    resp           = ff.lp_filter(np.nanmean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
    resp_BL        = np.zeros((n, np.sum(abs(dur))*Fs))
    #resp           = np.zeros((n, np.sum(abs(dur))*Fs))
    for k in range(n):
        resp_BL[k,:] = ff.lp_filter(np.nanmean(EEG_resp[rc,np.random.choice(stimNum_BL, len(stimNum_all)),:],0),45,Fs)
        #resp[k,:] = ff.lp_filter(np.nanmean(EEG_resp[rc,np.random.choice(stimNum_all, j),:],0),45,Fs)
    LL_BL     = get_LL_all(np.expand_dims(resp_BL, axis=0), Fs, w, 1, 0)
    #LL_resp   = get_LL_all(np.expand_dims(resp, axis=0), Fs, w, 1, 0)
    LL_resp   = get_LL_all(np.expand_dims(np.expand_dims(resp, axis=0),0), Fs, w, 1, 0)
    LL_BL     = LL_BL[0]
    LL_resp   = LL_resp[0,0]

    #####figure 
    fig   = plt.figure(figsize=(10,10) )
    plt.suptitle(labels_all[sc]+' -- '+labels_all[rc]+', Distance: '+str(np.round(lists.d.values[0],2))+'mm')
    gs    = fig.add_gridspec(2,1)  # GridSpec(4,1, height_ratios=[1,2,1,2])
    ax   = fig.add_subplot(gs[0,0])

    plt.xlim([-1, 1])
    plt.axvline(0, c=[0,0,0])
    plt.axvspan(0.015, 0.015+w, alpha=0.2, color=[1,0,0])
    plt.axvspan(0.015+w/2,0.015+3*w/2, color=[1,0.5,0], alpha=0.1)
    for k in range(n):
        plt.plot(x_ax,resp_BL[k], c=color_elab[0],alpha=0.7, linewidth=3, label='time surr '+str(k))
        #plt.plot(x_ax,resp[k], c=[0,0,0], linewidth=3, label='mean resp, n:'+str(len(stimNum_all)))
    plt.plot(x_ax,resp, c=[0,0,0], linewidth=3, label='mean resp, n:'+str(len(stimNum_all)))
    plt.title('Mean Responses')
    plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
    plt.ylim([-np.max([100,np.max(abs(resp))*1.1]),np.max([100,np.max(abs(resp))*1.1])])
    #plt.ylim([-100,100])
    plt.ylabel('[uV/ms]')
    ax   = fig.add_subplot(gs[1,0], sharex=ax)
    plt.axvline(0, c=[0,0,0])
    #plt.axvline(0+0.01+w/2, c=[0,0,0])
    #plt.axvspan(0.01+w/2, resp, alpha=0.8, color=color_elab[1])

    for k in range(n):
        plt.plot(x_ax,LL_BL[k], c=color_elab[0],alpha=0.7, linewidth=3)
        #plt.plot(x_ax,LL_resp[k], c=[0,0,0], linewidth=3)
    plt.plot(x_ax,LL_resp, c=[0,0,0], linewidth=3)
    #plt.plot(np.argmax(LL_resp[np.int64(Fs*(1.01+w/2)):np.int64(Fs*(1.01+1.5*w))])/Fs+0.01+w/2,np.max(LL_resp[np.int64(Fs*(1.01+w/2)):np.int64(Fs*(1.01+1.5*w))]), 'or')
    plt.title('LL of Mean Responses')
    #plt.axhline(np.percentile(LL_BL[:,0:np.int64(2*Fs)],99), c=[1,0,0], label ='99th percentile, entire 2s')#0:np.int64((1-w/2)*Fs)
    LL_meregd = np.concatenate([LL_BL, np.expand_dims(LL_resp,0)])
    #plt.axhline(np.percentile(LL_meregd[:,0:np.int64((2)*Fs)],99), c=[0.8,0,1], label ='99th percentile')#0:np.int64((1-w/2)*Fs)
    #plt.axhline(np.percentile(LL_resp[0:np.int64((1-w/2)*Fs)],99), c=[0.8,0,1], label ='99th percentile')#0:np.int64((1-w/2)*Fs)
    
    
    plt.ylim([0,np.max([2,LL_resp[np.int64(Fs*(1.01+w/2))]*1.1])])
    #plt.ylim([0,1.5])
    plt.ylabel(str(w)+'s LL [uV/ms]')

    thr        = np.percentile(LL_meregd[0:np.int64((1-w/2-0.05)*Fs)],99)
    plt.axhline(thr, c=[1,0,0], label ='99th percentile')#0:np.int64((1-w/2)*Fs)
    plt.legend()
    plt.axvline(0.01+w/2, c=[1,0,0], alpha=0.8)
    plt.axvline(0.01+w, c=[1,0.5,0], alpha=0.8)
    #plt.axvspan((0.015+0.5*w), (0.015+1*w), alpha=0.2, color=[1,0,0])
    LL_meregd = np.concatenate([LL_BL, np.expand_dims(LL_resp,0)])
    thr        = np.percentile(LL_meregd[0:np.int64((1-w/2-0.05)*Fs)],99)
    if  np.mean(LL_resp[np.int64((1.01+w/2)*Fs):np.int64((1+w)*Fs)]>thr)>0.9:
        print('yes')

In [ ]:
def get_LL_all(data, Fs,win, t0, IPI):  # specific for Int and IPI
    t1                  = np.int64(t0 * Fs) #start conditioning stim, time zero
    t2                  = np.int64((t0 + IPI / 1000) * Fs)#start probing stim
    #blank out stim artifact
    #data[:, :, np.int(t1 - 0.002 * Fs):np.int(t1 + 0.01 * Fs)] = 0
    #data[:, :, np.int(t2 - 0.002 * Fs):np.int(t2 + 0.01 * Fs)] = 0
    #wdp         = np.int(Fs * wdp_S)  # sliding window size in samples
    wdp         = np.int64(Fs * win)  # 100ms -> 50 sample points
    EEG_pad     = np.pad(data, [(0, 0), (0, 0), (np.int64(wdp / 2), np.int64(wdp / 2))], 'reflect')  # (18, 3006)
    LL_trial    = np.zeros((data.shape[0], data.shape[1], data.shape[2]))
    LL_max      = np.zeros((data.shape[0], data.shape[1], 2))
    for i in range(data.shape[2]):  # entire response
        n                       = i + np.int64(wdp / 2)
        LL_trial[:, :, i]       = np.nansum(abs(np.diff(EEG_pad[:, :, n - np.int64(wdp / 2):n + np.int64(wdp / 2)], axis=-1)),
                                   axis=-1)/(win*1000)
 # onset in ms after probing pulse, must be at least 10ms + half of the sliding window to not include fake data
    return LL_trial

### pk of mean method

In [ ]:
# sign resp channels
# LL_CCEP= LL_CCEP.drop(columns='RespC')
LL_CCEP.insert(0,'RespC', 0)
t0    = 1
w     = 0.2
Stims = np.unique(LL_CCEP.Stim)
for i in range(len(Stims)):
    sc = Stims[i]
    for rc in range(len(labels_all)):
        lists          = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==sc)]
        lists          = lists[~np.isnan(lists.zLL.values)]
        stimNum_all    = lists.Num.values.astype('int')
        if len(stimNum_all)>0:
            resp           = ff.lp_filter(np.nanmean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
            resp_BL = np.zeros((n, np.sum(abs(dur))*Fs))
            for k in range(n):
                resp_BL[k,:] = ff.lp_filter(np.nanmean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
            pk             = np.max(abs(resp[np.int64((t0+0.015)*Fs):np.int64((t0+w)*Fs)]))
            BL_m           = np.nanmean(resp[0:np.int64((t0-0.1)*Fs)])
            BL_s           = np.nanstd(resp[0:np.int64((t0-0.1)*Fs)])
            if pk > BL_m+6*BL_s:
                LL_CCEP.loc[(LL_CCEP.Chan ==rc)&(LL_CCEP.Stim==sc), 'RespC'] = 1       

In [ ]:
dat[(dat.Num==stimNum)&(dat.Chan== rc)]

In [ ]:
### single trial example

r = 1 #&(LL_CCEP['LL'] <1)
a = 0
dat      = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP['d']>70)&(LL_CCEP['LL']>10)&(LL_CCEP['RespC'] ==1)]

k        = np.random.randint(0, len(dat))
stimNum  = np.int64(dat.Num.values[k])
Stim_chs = np.int64(dat.Stim.values[k])

ChanP = StimChanSM[np.array(np.where(np.array(StimChans) == labels_all[Stim_chs]))[0,0]]
rc    = np.int64(dat.Chan.values[k])
# find other stimulations
lists = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==Stim_chs)]
lists = lists[~np.isnan(lists.zLL.values)]

fig   = plt.figure(figsize=(10,5) )
#plt.title(labels_all[Stim_chs]+' -- '+labels_clinic[rc])
plt.title(labels_all[Stim_chs]+' -- '+labels_all[rc]+', Dist: '+str(np.round(dat.d.values[k]))+'mm')



plt.xlim([-0.6,1])
plt.ylim([-np.max([np.max(EEG_resp[rc,stimNum,:])*1.1,800]),np.max([np.max(EEG_resp[rc,stimNum,:])*1.1,500])])
plt.axvline(0, c=[0,0,0])
plt.xlabel('time [s]')
plt.text(-0.3, 200, 'rLL: '+str(np.round(dat.rLL.values[k],2)), c=[1-r,0,0])
plt.text(-0.3, 300, 'zLL: '+str(np.round(dat.zLL.values[k],2)), c=[0,0,0])
plt.text(-0.3, 100, 'LL: '+str(np.round(dat.LL.values[k],2)), c=[0,0,0])


#stim_spec                = stimlist[(stimlist.condition>0)&(stimlist.ChanP==ChanP)&(stimlist.IPI_ms ==0)&(stimlist.noise ==0)]#&(stimlist.noise ==0)
stimNum_all                  = lists.Num.values.astype('int')
plt.plot(x_ax,ff.lp_filter(np.mean(EEG_resp[rc,stimNum_all,:],0),45,Fs), c=[0,0,0], linewidth=1, label='mean, n='+str(len(stimNum_all)))
plt.plot(x_ax,ff.lp_filter(EEG_resp[rc,stimNum,:],45,Fs), c=color_elab[0], linewidth=2, label='trial')
plt.axvspan(t_0+w_r-1, t_0-1+w_LL, alpha=0.3, color=color_elab[0])
plt.axvspan(t_0+0.015-1, t_0+w_r-1, alpha=0.8, color=color_elab[1])
plt.axvspan(t_0+0.015-1-0.2, t_0+w_r-1-0.2, alpha=0.8, color=color_elab[1])
plt.legend()
plt.show()


In [ ]:
##  butterlfy example
rc =28
Stim_chs = 36#rc-2# 22
lists = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==Stim_chs)]
lists = lists[~np.isnan(lists.zLL.values)]
fig   = plt.figure(figsize=(10,7))
plt.title(labels_clinic[Stim_chs]+' -- '+labels_clinic[rc])
#plt.title(labels_all[Stim_chs]+' -- '+labels_clinic[rc]+', Dist: '+str(np.round(dat.d.values[k]))+'mm')



plt.xlim([-0.2,0.3])
plt.ylim([-np.max([np.min([np.max(EEG_resp[rc,stimNum,:])*1.1,900]),300]),np.max([np.min([np.max(EEG_resp[rc,stimNum,:])*1.1,900]),300])])

plt.xlabel('time [s]')
#plt.text(-0.6, 200, 'rLL: '+str(np.round(dat.rLL.values[k],2)), c=[1-r,0,0])
#plt.text(-0.6, 300, 'zLL: '+str(np.round(dat.zLL.values[k],2)), c=[1-a,0,0])
axes = plt.gca()

#stim_spec                = stimlist[(stimlist.condition>0)&(stimlist.ChanP==ChanP)&(stimlist.IPI_ms ==0)&(stimlist.noise ==0)]#&(stimlist.noise ==0)
stimNum_all                  = lists.Num.values.astype('int')
for i in range(len(stimNum_all)):
        plt.plot(x_ax,ff.lp_filter(EEG_resp[rc,stimNum_all[i],:],45,Fs), c=color_elab[1], linewidth=1)
plt.plot(x_ax,ff.lp_filter(np.mean(EEG_resp[rc,stimNum_all,:],0),45,Fs), c=[0,0,0], linewidth=3)
#plt.text(-0.7, axes.get_ylim()[0]*2/3, 'n: '+str(len(stimNum_all)))
plt.axvline(0, c=[0,0,0])
plt.axvline(0.015, c=[0,0,0], linewidth=0.2)

plt.show()

In [ ]:
## single trial and mean
r =0 #&(LL_CCEP['LL'] <1)
d = 8
ll = 0
dat      = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP['RespC'] ==1)&(LL_CCEP['LL'] >ll)&(LL_CCEP['d'] >d)&(LL_CCEP['RespR'] ==r)]

k        = np.random.randint(0, len(dat))
stimNum  = np.int64(dat.Num.values[k])
Stim_chs = np.int64(dat.Stim.values[k])

ChanP = StimChanSM[np.array(np.where(np.array(StimChans) == labels_all[Stim_chs]))[0,0]]
rc    = np.int64(dat.Chan.values[k])
# find other stimulations
lists = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP['Stim']==Stim_chs)]
lists = lists[~np.isnan(lists.zLL.values)]
stimNum_all                  = lists.Num.values.astype('int')
resp = ff.lp_filter(np.mean(EEG_resp[rc,stimNum_all,:],0),45,Fs)
fig   = plt.figure(figsize=(10,10) )
plt.suptitle(labels_all[Stim_chs]+' -- '+labels_all[rc]+', Dist: '+str(np.round(dat.d.values[k]))+'mm')
gs    = fig.add_gridspec(2,1)  # GridSpec(4,1, height_ratios=[1,2,1,2])
ax   = fig.add_subplot(gs[0,0])

plt.xlim([-1,0.5])
plt.ylim([-np.min([np.max([np.max(abs(EEG_resp[rc,stimNum,:]))*1.1,400]), 800]),np.min([np.max([np.max(abs(EEG_resp[rc,stimNum,:]))*1.1,400])])])
plt.axvline(0, c=[0,0,0])
plt.text(-0.7, 200, 'rLL: '+str(np.round(dat.rLL.values[k],2)), c=[1-r,0,0])
plt.text(-0.7, 300, 'zLL: '+str(np.round(dat.zLL.values[k],2)), c=[0,0,0])
plt.text(-0.7, 100, 'LL: '+str(np.round(dat.LL.values[k],2)), c=[0,0,0])
plt.axvspan(t_0+w_r-1, t_0-1+w_LL, alpha=0.3, color=color_elab[0] )
plt.axvspan(t_0+0.015-1, t_0+w_r-1, alpha=0.8, color=color_elab[1])
plt.axvspan(t_0+0.015-1-0.2, t_0+w_r-1-0.2, alpha=0.8, color=color_elab[1])
plt.plot(x_ax,ff.lp_filter(EEG_resp[rc,stimNum,:],45,Fs), c=color_elab[0], linewidth=3)
plt.plot(x_ax,resp, c=[0,0,0],alpha=0.7, linewidth=3, label='mean, n='+str(len(stimNum_all)))
plt.legend()
plt.title('Trial')

#plt.title(labels_all[Stim_chs]+' -- '+labels_clinic[rc]+', Dist: '+str(np.round(dat.d.values[k]))+'mm')


ax   = fig.add_subplot(gs[1,0], sharex=ax)
plt.xlabel('time [s]')
axes = plt.gca()
plt.title('MEAN')
plt.plot(x_ax,resp, c=[0,0,0],alpha=0.7, linewidth=3, label = 'mean resp')
plt.plot(x_ax,abs(resp), c=[0,0,0], linewidth=1, label = 'abs mean resp')
plt.axhline(np.max(abs(resp)[np.int64((t0+0.015)*Fs):np.int64((t0+0.2)*Fs)]), c=color_elab[0], label = 'peak resp')
plt.axhline(np.mean(resp[0:np.int64((t0-0.1)*Fs)])+6*np.std(abs(resp)[0:np.int64((t0-0.1)*Fs)]), c=[1,0,0], label = 'thr: zscore of 6')
plt.axhline(np.mean(resp[0:np.int64((t0-0.1)*Fs)]), alpha=0.1, c=[1,0,0])

#plt.text(-0.7, axes.get_ylim()[0]*2/3, 'n: '+str(len(stimNum_all)))
plt.axvline(0, c=[0,0,0])
plt.legend()
#plt.ylim([-100, 100])
plt.show()

In [ ]:
stop

## PCIst

https://github.com/renzocom/PCIst

In [ ]:
par      = {'baseline_window':(-0.5, -0.02), 'response_window':(0.02,0.5), 'k':1.2, 'min_snr':1.1, 'max_var':99, 'embed':False,'n_steps':5} #
hour     = np.unique(LL_CCEP.Hour)
M_PCI    = np.zeros((len(np.unique(LL_CCEP.Stim)), 24))
resp_all = np.zeros((len(np.unique(LL_CCEP.Stim)), len(labels_all),24, 2000))
for i in range(24):
    for j in range(len(np.unique(LL_CCEP.Stim))):
        sc      = np.int64(np.sort(np.unique(LL_CCEP.Stim))[j])
        
        lists     = LL_CCEP[(LL_CCEP['Stim']==sc)&(LL_CCEP['Hour']==i)]
        if len(lists)>0:
            list_nan  = lists[np.isnan(lists.LL.values)]
            stimNum   = np.int64(np.unique(lists.Num))
            resp      = ff.lp_filter(EEG_resp[:,stimNum,:],40,Fs)
            # remove bad combinations
            for k in range(len(list_nan)):
                n = np.int64(list_nan.Num.values[k])
                n = np.where(stimNum== n)[0][0]
                resp[np.int64(list_nan.Chan.values[k]),n,:] = np.nan
            if len(resp)>0:
                mean_resp          = np.nanmean(resp, 1)
                resp_all[j,:,i,:]  = mean_resp
                if np.nanmean(abs(mean_resp))>0: 
                    PCIst              =  pci.calc_PCIst(np.nan_to_num(mean_resp), np.arange(-1,3, 1/Fs), **par)
                    M_PCI[j,i]         = PCIst
                else:# if all data are nan
                    M_PCI[j,i]         = -1
        else:
                M_PCI[j,i]         = -1
                resp_all[j,:,i,:]  = np.nan
                
labels_stims = labels_all[np.int64(np.sort(np.unique(LL_CCEP.Stim)))]

In [ ]:
np.save(path_patient + '/Analysis/BrainMapping/PCI/PCI_h.npy', M_PCI)

In [ ]:
try:
    os.mkdir(path_patient + '/Analysis/BrainMapping/PCI/figures/')
    os.mkdir( path_patient + '/Analysis/BrainMapping/PCI/')
    
except OSError:
    print("test already exists")
filename     = path_patient + '/Analysis/BrainMapping/PCI/figures/PCI_hours.png'


In [ ]:
sc = 73
fig, axs = plt.subplots(4,6, figsize=(25,15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .1, wspace=.1)
plt.suptitle(labels_stims[sc])
axs = axs.ravel()
for i in range(24):
    axs[i].set_title(str(i)+':00, PCI: '+str(np.round(M_PCI[sc,i])))
    axs[i].set_yticks([])
    axs[i].set_xticks([])
    axs[i].set_ylim(-700, 700)
    axs[i].set_xlim(-0.5, 1)
    axs[i].axvline(0, c=[0,0,0])
    for c in range(len(labels_all)):
        axs[i].plot(x_ax,resp_all[sc,c,i,:] )


In [ ]:
filename     = path_patient + '/Analysis/BrainMapping/PCI/figures/PCI_hours.png'

# Plot distance matrix.
#cmap     = ListedColormap(['r', 'k', 'w'])
fig      = pylab.figure(figsize=(15,15))
axmatrix = fig.add_axes([0.15,0.05,0.7,0.9])#fig.add_axes([0.4,0.1,0.6,0.6])

im = axmatrix.matshow(M_PCI, aspect='auto', origin='lower',cmap='hot', vmin=np.percentile(M_PCI,30), vmax=np.percentile(M_PCI,95))

plt.xticks(np.arange(0, 24, 6));
plt.xlabel('Time of the day')
plt.yticks(range(len(StimChans)), labels_stims);
#plt.axvline(15, c=[0,0,0], linewidth=35)
# Plot colorbar.
axcolor = fig.add_axes([0.9,0.05,0.01,0.9])
pylab.colorbar(im, cax=axcolor)
plt.savefig(filename)


## CCEP CR

In [ ]:
LL_CCEP = pd.read_csv(path_patient + '/Analysis/BrainMapping/LL/LL_CCEP_'+str(exp)+'_'+str(w)+'s.csv')

In [ ]:
labels_all[StimChanNums]

In [ ]:
cs = np.unique(LL_CCEP.loc[(LL_CCEP.LL >0)&(LL_CCEP.RespC ==1)&(LL_CCEP.Stim ==s), 'Chan'])

In [ ]:
c

In [ ]:
hours    = np.unique(LL_CCEP.Hour)
StimChanNums = np.sort(StimChanIx)
M_hour   = np.zeros((len(labels_all), len(labels_all), len(hours)))
M_resp   = np.zeros((len(labels_all), len(labels_all)))

for sc in range(len(StimChans)):
    s      = StimChanNums[sc]
    dat    = LL_CCEP.loc[(LL_CCEP.LL >0)&(LL_CCEP.RespC ==1)&(LL_CCEP.Stim ==s)]
    cs     = np.unique(LL_CCEP.loc[(LL_CCEP.LL >0)&(LL_CCEP.RespC ==1)&(LL_CCEP.Stim ==s), 'Chan'])
    if len(dat)>0:
        for c in cs:
            c = np.int64(c)
            for hs in range(len(hours)):
                h              = hours[hs]
                M_hour[s,c,hs] = np.nanmean(dat.loc[(dat.Hour ==h)&(dat.Chan ==c), 'LL'].values)
            
M_hour=np.nan_to_num(M_hour, -2)
M_hour[:,bad_chans,:] = -2
M_resp[:,bad_chans] = 0


In [ ]:
##test single plot
h        = 3
fig      = pylab.figure(figsize=(20,20))
axmatrix = fig.add_axes([0.1,0.1,0.9,0.9])

im = axmatrix.matshow(M_hour[:,:,h], aspect='auto', origin='lower', cmap='hot', vmin=1, vmax=10)

plt.xticks(range(len(labels_all)), labels_all, rotation=90)
plt.yticks(range(len(labels_all)), labels_all)
plt.xlabel(str(h)+':00', fontsize=16)
plt.show()

In [ ]:
os.mkdir(path_fig)

In [ ]:
## GIF
path_fig   = path_patient + '/Analysis/BrainMapping/LL/figures/LL_dist'
hours      = np.unique(LL_CCEP.Hour)
n_frames   = len(hours)
gif_name   = 'scatter'

print('Creating charts\n')
filenames = []

for i in np.arange(n_frames):
    # plot
    #fig, ax    = plt.subplots(figsize=(12, 8))
    fig   = plt.figure(figsize=(15,8) )
    gs    = fig.add_gridspec(1,2, width_ratios=[20,1])  # GridSpec(4,1, height_ratios=[1,2,1,2])
    ax   = fig.add_subplot(gs[0,0])
    #ax.set_facecolor(bg_color)
    data     = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP.Hour ==hours[i])&(LL_CCEP.d >7.5)&(LL_CCEP['RespR'] ==1)&(LL_CCEP['RespC'] ==1)]
    #dataR    = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP.Hour ==hours[i])&(LL_CCEP.d >7.5)&(LL_CCEP['RespR'] ==0)&(LL_CCEP['RespC'] ==1)]
    data0    = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP.Hour ==hours[i])&(LL_CCEP.d >7.5)&(LL_CCEP['RespC'] ==0)]
    n_all  = len(LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP.Hour ==hours[i])&(LL_CCEP.d >7.5)])
    n_resp = len(data)
    #axs.scatter(data.Distance, data['LL norm'], 5, c="g", alpha=1, label = 'LL')
    plt.plot(data0.d, data0['LL'], 'o', c=[0,0,0], label = 'non-resp channel', mfc='none', alpha=0.25)
    plt.plot(dataR.d, dataR['LL'], 'o', c=color_elab[1], label = 'non-resp ratio', alpha=0.45)
    plt.plot(data.d, data['LL'], 'o', c=color_elab[0], label = 'resp')
    plt.ylim([0,20])
    plt.xlim([0,120])
    plt.xlabel('distance [mm]')
    plt.ylabel('LL [uV/ms]')
    plt.legend()
    plt.title(str(np.int(hours[i]))+':00 --- Resp = '+str(np.round(n_resp/n_all,4)*100)+'%')
    
    # remove spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # grid
    ax.set_axisbelow(True)
    ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)
    ax = fig.add_subplot(gs[0,1])
    n_stim = len(data)
    plt.title('number of valid recordings: '+str(n_all))
    plt.bar(1,n_resp/n_all*100+len(dataR)/n_all*100+len(data0)/n_all*100, color=[0,0,0])
    plt.bar(1,n_resp/n_all*100+len(dataR)/n_all*100, color=color_elab[1])
    plt.bar(1,n_resp/n_all*100, color=color_elab[0])
    plt.ylim([0,100])
    plt.xlim([0.5,1.5])
    plt.xticks([])
    plt.ylabel('%')
    # build file name and append to list of file names
    filename = path_fig+'/frame_'+str(i)+'.png'
    filenames.append(filename)

    # last frame of each viz stays longer
    if (i == n_frames):
        for i in range(5):
            filenames.append(filename)
    # save img
    plt.savefig(filename)
    plt.close()
print('Charts saved\n')
# Build GIF
print('Creating gif\n')
with imageio.get_writer(path_fig+'/scatter.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
print('Gif saved\n')
print('Removing Images\n')
 # Remove files
for filename in set(filenames):
    os.remove(filename)
print('DONE')

In [ ]:
## GIF BM
path_fig  = path_patient + '/Analysis/BrainMapping/LL/figures/BM_plot'
n_frames   = len(hours)
if not os.path.exists(path_fig):
    os.makedirs(path_fig)
    
print('Creating charts\n')
filenames = []
for i in np.arange(n_frames):
    fig      = pylab.figure(figsize=(20,20))
    axmatrix = fig.add_axes([0.1,0.1,0.8,0.8])

    im = axmatrix.matshow(M_hour[:,:,i], aspect='auto', origin='lower', cmap='hot', vmin=0.5, vmax=10)

    plt.xticks(range(len(labels_all)), labels_all, rotation=90)
    plt.yticks(range(len(labels_all)), labels_all)
    plt.xlabel(str(i)+':00', fontsize=16)

    # build file name and append to list of file names
    filename = path_fig+'/frame_'+str(i)+'.png'
    filenames.append(filename)

    # save img
    plt.savefig(filename)
    plt.close()
print('Charts saved\n')
# Build GIF
print('Creating gif\n')
with imageio.get_writer(path_fig+'/BM_LL.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
print('Gif saved\n')
print('Removing Images\n')
 # Remove files
for filename in set(filenames):
    os.remove(filename)
print('DONE')

In [ ]:
sc     = 8
rc     =  5
plt.figure(figsize=(12,8))
plt.title(labels_all[sc]+'-->'+labels_all[rc])
# stim_spec                = stimlist[(stimlist.ChanP==ChanP)&(stimlist.IPI_ms ==0)&(stimlist.noise ==0)]#&(stimlist.noise ==0)
# stimNum_all                  = stim_spec.StimNum.values#[:,0]

lists     = LL_CCEP[(LL_CCEP['Stim']==sc)&(LL_CCEP['Chan']==rc)]
lists  = lists[~np.isnan(lists.LL.values)]
stimNum   = np.int64(np.unique(lists.Num))
            
color_time = np.zeros((len(stimNum),3))
color_time[:,0] = np.linspace(0,1,len(stimNum))

for t in range(len(stimNum)):
    plt.plot(x_ax,ff.lp_filter(EEG_resp[rc,stimNum[t],:], 30, Fs), c=color_time[t], linewidth=1)
plt.plot(x_ax,np.mean(EEG_resp[rc,stimNum,:],0), c=[0,0,0], linewidth=3)
plt.xlim([-0.5,1])
#plt.ylim([-400,400])
plt.axvline(0, c=[0,0,0], linewidth=5)
plt.xlabel('time [s]')
plt.ylim([-800,800])
plt.tight_layout()

In [ ]:
dat   = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP.d >60)&(LL_CCEP['RespC']==1)]

k       = np.random.randint(0, len(dat))
stimNum = np.int(dat.Num.values[k])

Stim_chs = np.int(dat.Stim.values[k])
ChanP = StimChanNums[np.array(np.where(np.array(StimChans) == labels_all[Stim_chs]))[0,0]]
rc = np.int(dat.Chan.values[k])
    
fig   = plt.figure(figsize=(8,8) )
plt.suptitle(labels_all[Stim_chs]+' -- '+labels_all[rc]+', Dist: '+str(np.round(dat.d.values[k]))+'mm')
gs    = fig.add_gridspec(2,1)  # GridSpec(4,1, height_ratios=[1,2,1,2])
axs   = fig.add_subplot(gs[0,0])
CCEP_mean = EEG_resp[rc,stimNum,:]
#plt.plot(x_ax,CCEP_mean, c=color_elab[1], linewidth=1.5)
plt.plot(x_ax,CCEP_mean, c=color_elab[0], linewidth=2.5,alpha=0.5,  label = 'org')
plt.plot(x_ax,ff.lp_filter(CCEP_mean, 45,Fs), c=color_elab[2], linewidth=1.5, label = 'LP45Hz')
plt.plot(x_ax,ff.lp_filter(CCEP_mean, 30,Fs), c=[0,0,0], linewidth=1.5, label = 'LP30Hz')
plt.text(-0.3, 200, 'LL: '+str(np.round(dat.LL.values[k],2)))
plt.text(-0.3, 100, 'zLL: '+str(np.round(dat['zLL'].values[k],2)))
plt.text(-0.3, 300, 'rLL: '+str(np.round(dat['rLL'].values[k],2)))
#plt.xlim([-0.5,1])
#plt.ylim([-400,400])
plt.axvline(0, c=[0,0,0])
plt.xlim([-0.4,1])
plt.title('single trial')
plt.legend()

axs   = fig.add_subplot(gs[1,0], sharex=axs, sharey=axs)
stim_spec                = stimlist[(stimlist.ChanP==ChanP)&(stimlist.IPI_ms ==0)&(stimlist.noise ==0)]#&(stimlist.noise ==0)
stimNum_all                  = stim_spec.StimNum.values#[:,0]

for t in range(len(stimNum_all)):
    plt.plot(x_ax,ff.lp_filter(EEG_resp[rc,stimNum_all[t],:], 30, Fs), c=color_elab[0], linewidth=1)
plt.plot(x_ax,np.mean(EEG_resp[rc,stimNum_all,:],0), c=[0,0,0], linewidth=3)
plt.text(-0.3, 100, 'zLL: '+str(np.round(np.mean(LL_CCEP[(LL_CCEP.Chan==rc)&(LL_CCEP.Stim==Stim_chs)]['zLL']),2)))
#plt.xlim([-0.5,1])
#plt.ylim([-400,400])
plt.axvline(0, c=[0,0,0])
plt.title('all trials')
plt.xlabel('time [s]')
#plt.ylim([-15,15])

## Asymmetry

In [ ]:
stim_av = StimChanIx
for i in range(len(bad_chans)):
    stim_av = np.delete(stim_av, np.where(stim_av==bad_chans[i]))

In [ ]:
data_A

In [ ]:
data_test = data_A[(data_A.R>1.2)&(data_A.d>7.5)]   
#data_test = data_A[(data_A.R>0.95)&(data_A.R<1.05)&(data_A.d>7.5)]   
k = np.random.choice(len(data_test))
c1 = np.int64(data_test.Stim.values[k])
c2 = np.int64(data_test.Chan.values[k])
lists     = LL_CCEP[(LL_CCEP['Stim']==c1)&(LL_CCEP['Chan']==c2)]
lists     = lists[~np.isnan(lists.LL.values)]
stimNum   = np.int64(np.unique(lists.Num))
fig = plt.figure(figsize=(15, 6))
gs    = fig.add_gridspec(1,2)  # GridSpec(4,1, height_ratios=[1,2,1,2])
plt.suptitle(labels_all[c1]+'<-->'+labels_all[c2]+', Dist: '+str(np.round(np.mean(lists.d),1))+'mm')

##plot 1
ax         = fig.add_subplot(gs[0,0])

            
color_time = np.zeros((len(stimNum),3))
color_time[:,0] = np.linspace(0,1,len(stimNum))
plt.title('mean LL: '+str(np.round(np.mean(lists.LL),1))+' uV/ms')
for t in range(len(stimNum)):
    plt.plot(x_ax,ff.lp_filter(EEG_resp[c2,stimNum[t],:], 30, Fs), c=color_time[t], linewidth=1)
plt.plot(x_ax,np.mean(ff.lp_filter(EEG_resp[c2,stimNum,:], 30, Fs),0), c=[0,0,0], linewidth=3)
plt.xlim([-0.3,0.7])
#plt.ylim([-400,400])
plt.axvline(0, c=[0,0,0], linewidth=5)
plt.xlabel('time [s]')
plt.text(0.4, 600, 'n: '+str(len(stimNum)))
plt.ylim([-800,800])

##plot 2
ax   = fig.add_subplot(gs[0,1], sharex = ax, sharey=ax)
lists     = LL_CCEP[(LL_CCEP['Stim']==c2)&(LL_CCEP['Chan']==c1)]
lists  = lists[~np.isnan(lists.LL.values)]
stimNum   = np.int64(np.unique(lists.Num))
            
color_time = np.zeros((len(stimNum),3))
color_time[:,0] = np.linspace(0,1,len(stimNum))
plt.title('mean LL: '+str(np.round(np.mean(lists.LL),1))+' uV/ms')
for t in range(len(stimNum)):
    plt.plot(x_ax,ff.lp_filter(EEG_resp[c1,stimNum[t],:], 30, Fs), c=color_time[t], linewidth=1)
plt.plot(x_ax,np.mean(ff.lp_filter(EEG_resp[c1,stimNum,:], 30, Fs),0), c=[0,0,0], linewidth=3)

plt.text(0.4, 600, 'n: '+str(len(stimNum)))
plt.axvline(0, c=[0,0,0], linewidth=5)
plt.xlabel('time [s]')

plt.show()



In [ ]:
c1 = 6#np.random.choice(stim_av)
c2 = 16 #np.random.choice(stim_av)

lists     = LL_CCEP[(LL_CCEP['Stim']==c1)&(LL_CCEP['Chan']==c2)]
lists     = lists[~np.isnan(lists.LL.values)]
stimNum   = np.int64(np.unique(lists.Num))
fig = plt.figure(figsize=(15, 6))
gs    = fig.add_gridspec(1,2)  # GridSpec(4,1, height_ratios=[1,2,1,2])
plt.suptitle(labels_all[c1]+'<-->'+labels_all[c2]+', Dist: '+str(np.round(np.mean(lists.d),1))+'mm')

##plot 1
ax         = fig.add_subplot(gs[0,0])

            
color_time = np.zeros((len(stimNum),3))
color_time[:,0] = np.linspace(0,1,len(stimNum))
plt.title('mean LL: '+str(np.round(np.mean(lists.LL),1))+' uV/ms')
for t in range(len(stimNum)):
    plt.plot(x_ax,ff.lp_filter(EEG_resp[c2,stimNum[t],:], 30, Fs), c=color_time[t], linewidth=1)
plt.plot(x_ax,np.mean(ff.lp_filter(EEG_resp[c2,stimNum,:], 30, Fs),0), c=[0,0,0], linewidth=3)
plt.xlim([-0.3,0.7])
#plt.ylim([-400,400])
plt.axvline(0, c=[0,0,0], linewidth=5)
plt.xlabel('time [s]')
plt.text(0.4, 600, 'n: '+str(len(stimNum)))
plt.ylim([-800,800])

##plot 2
ax   = fig.add_subplot(gs[0,1], sharex = ax, sharey=ax)
lists     = LL_CCEP[(LL_CCEP['Stim']==c2)&(LL_CCEP['Chan']==c1)]
lists  = lists[~np.isnan(lists.LL.values)]
stimNum   = np.int64(np.unique(lists.Num))
            
color_time = np.zeros((len(stimNum),3))
color_time[:,0] = np.linspace(0,1,len(stimNum))
plt.title('mean LL: '+str(np.round(np.mean(lists.LL),1))+' uV/ms')
for t in range(len(stimNum)):
    plt.plot(x_ax,ff.lp_filter(EEG_resp[c1,stimNum[t],:], 30, Fs), c=color_time[t], linewidth=1)
plt.plot(x_ax,np.mean(ff.lp_filter(EEG_resp[c1,stimNum,:], 30, Fs),0), c=[0,0,0], linewidth=3)

plt.text(0.4, 600, 'n: '+str(len(stimNum)))
plt.axvline(0, c=[0,0,0], linewidth=5)
plt.xlabel('time [s]')

plt.show()



In [ ]:
data_asym     = np.zeros((1,6))
lists         = LL_CCEP[(LL_CCEP['RespC']==1)]
lists         = lists[~np.isnan(lists.LL.values)]
for s in range(len(labels_all)):
    for c in range(len(labels_all)):
        list_1 = lists[(lists.Stim==s)&(lists.Chan==c)]
        list_2 = lists[(lists.Stim==c)&(lists.Chan==s)]
        if len(list_1)>0 and len(list_2)>0:
            val         = np.zeros((1, 6))
            val[0, 0]   = c                                         # response channel
            val[0, 1]   = s                                         # response channel
            val[0, 2]   = np.nanmean(list_1.LL)/np.nanmean(list_2.LL)
            val[0, 3]   = np.nanmean(list_1.d)
            data_asym    = np.concatenate((data_asym, val), axis=0)
data_asym = data_asym[1:-1, :] # remove first row (dummy row)
data_A = pd.DataFrame(
    {"Chan": data_asym[:, 0], "Stim": data_asym[:, 1], "R": data_asym[:, 2], "d": data_asym[:, 3]})

for c in range(len(labels_all)):
    data_A.loc[(data_A.Chan ==c), "Recs"]      = labels_all[c]
    data_A.loc[(data_A.Stim ==c), "Stim Region"]   = labels_region[c]
    data_A.loc[(data_A.Chan ==c), "Resp Region"]   = labels_region[c]
    data_A.loc[(data_A.Stim ==c), "Stims"]     = labels_all[c]

In [ ]:
data_A

In [ ]:
#fig, ax = plt.figure(figsize=(13,13))

fig, ax     = plt.subplots(figsize=(15,12))
RR = np.round(1/1.2,2)
plt.title(subj+' - Connections with Ratio < '+str(RR))

data_test = data_A[(data_A.R<RR)&(data_A.d>7.5)]        
for i in range(len(labels_all)):
    s = StimChanIx_sort[i]
    ax.add_patch(Rectangle((6.5, i-0.5), 1, 1, color = color_regions[np.where(regions==labels_region[s])[0][0]]))
    data_plot = data_test[data_test.Stim==s]
    for j in range(len(data_plot)):
        #plt.plot(data_plot.d.values[j], i,'o', markersize=10, c= color_regions[np.where(regions==data_plot['Resp Region'].values[j])[0][0]])
        plt.scatter(data_plot.d.values[j], i, s=80, c=color_regions[np.where(regions==data_plot['Resp Region'].values[j])[0][0]], alpha=0.8, edgecolors=[0,0,0])
    if any (region_border == i):
        num= region_num[np.where(region_border==i)[0][0]]
        t = plt.text(-10, (i+num/2)-1, v[i], fontsize=15)
        #t = plt.text(0.5, 0.5, 'text', transform=ax.transAxes, fontsize=30)
        t.set_bbox(dict(facecolor=color_regions[np.where(regions==labels_region[s])[0][0]], alpha=0.5, edgecolor=color_regions[np.where(regions==labels_region[s])[0][0]]))


plt.yticks(np.arange(len(labels_all)),labels_sort, fontsize=7)
plt.xlim([6.5, 80])
plt.ylim([-0.5, len(labels_all)-0.5])
plt.xlabel('euclidean distance [mm]')
plt.show()

In [ ]:
fig    = plt.figure(figsize=(12, 6))
data_A2 = data_A#[(data_A.Region1!='WM')&(data_A.Region2!='WM')]
#axs.scatter(data.Distance, data['LL norm'], 5, c="g", alpha=1, label = 'LL')
tissue = np.unique(data_A2.Region2)
for i in range(len(tissue)):
    data_A3 = data_A2[data_A2.Region2==tissue[i]]
    plt.plot(data_A3.d, data_A3['R'], 'o', alpha=0.8, label=tissue[i])

plt.xlabel('distance [mm]')
plt.ylabel('LL ratio')
plt.legend(title='Tissue2')
#plt.axhline(1,c=[0,0,0])
plt.ylim([1,5])


In [ ]:
fig    = plt.figure(figsize=(12, 6))
data_A2 = data_A#[(data_A.Region1!='WM')&(data_A.Region2!='WM')]
#axs.scatter(data.Distance, data['LL norm'], 5, c="g", alpha=1, label = 'LL')
tissue = np.unique(data_A2.Region1)
for i in range(len(tissue)):
    data_A3 = data_A2[data_A2.Region1==tissue[i]]
    plt.plot(data_A3.d, data_A3['R'], 'o', alpha=0.8, label=tissue[i])

plt.xlabel('distance [mm]')
plt.ylabel('LL ratio')
plt.legend(title='Tissue1')
#plt.axhline(1,c=[0,0,0])
plt.ylim([1,5])


In [ ]:
M_hourA        = np.zeros((len(labels_all), len(labels_all), 24))-1
# remove bad stims
lists         = LL_CCEP[(LL_CCEP['RespC']==1)]
lists         = lists[~np.isnan(lists.LL.values)]
for h in range(24):
    print(h)
    for s in range(len(labels_all)):
        for c in range(len(labels_all)):
            list_1 = lists[(lists.Stim==s)&(lists.Chan==c)&(lists.Hour==h)]
            list_2 = lists[(lists.Stim==c)&(lists.Chan==s)&(lists.Hour==h)]
            if len(list_1)>0 and len(list_2)>0:
                M_hourA[s,c,h] = np.nanmean(list_1.LL)/np.nanmean(list_2.LL)

In [ ]:
M_hourA[M_hourA==-1] =1

In [ ]:
## GIF BM
path_fig  = path_patient + '/Analysis/BrainMapping/LL/figures/BM_plot'
n_frames   = 24
if not os.path.exists(path_fig):
    os.makedirs(path_fig)
    
print('Creating charts\n')
filenames = []
for i in np.arange(n_frames):
    fig      = pylab.figure(figsize=(20,20))
    axmatrix = fig.add_axes([0.1,0.1,0.7,0.7])

    im = axmatrix.matshow(M_hourA[:,:,i], aspect='auto', origin='lower', cmap='seismic', vmin=0, vmax=2)

    plt.xticks(range(len(labels_all)), labels_all, rotation=90)
    plt.yticks(range(len(labels_all)), labels_all)
    plt.xlabel(str(i)+':00', fontsize=16)
    
    # Plot colorbar.
    axcolor = fig.add_axes([0.9,0.05,0.01,0.9])
    pylab.colorbar(im, cax=axcolor)

    # build file name and append to list of file names
    filename = path_fig+'/frame_'+str(i)+'.png'
    filenames.append(filename)

    # save img
    plt.savefig(filename)
    plt.close()
print('Charts saved\n')
# Build GIF
print('Creating gif\n')
with imageio.get_writer(path_fig+'/BM_asym.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
print('Gif saved\n')
print('Removing Images\n')
 # Remove files
#for filename in set(filenames):
#    os.remove(filename)
print('DONE')

## NETWORK PLOTS

In [ ]:
# libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import nxviz
from nxviz.plots import CircosPlot, MatrixPlot, ArcPlot, BasePlot

G.add_edge(1, 3, weight=7, capacity=15, length=342.7)
plt.figure(figsize=(15,15))
nx.draw(g, with_labels=True, alpha= 0.6, node_size=50)

In [ ]:
def make_graph(nodes_df, edges_df):
    g = nx.DiGraph()
    for i,row in nodes.iterrows():

        keys = row.index.tolist()

        values = row.values

        # The dict contains all attributes

        g.add_node(row['ID'], **dict(zip(keys,values)))


    for i,row in edges.iterrows():

        keys = row.index.tolist()

        values = row.values

        g.add_edge(row['source'], row['target'], weight=row['LL'],**dict(zip(keys,values)))

    return g

In [ ]:
edges

In [ ]:
data_test = LL_CCEP[(LL_CCEP.RespC==1)&(LL_CCEP.LL>1)&(LL_CCEP.d>50)]
data_test = data_test.drop(columns=['RespC', 'RespR', 'RespA','rLL', 'zLL', 'Day'])
data_test.insert(0,'Recs', 0)
data_test.insert(0,'Stims', 0)
for c in range(len(labels_all)):
    data_test.loc[(data_test.Stim ==c), "Region"] = labels_region[c]
    data_test.loc[(data_test.Chan ==c), "RegionC"] = labels_region[c]
    data_test.loc[(data_test.Chan ==c), "Recs"]   = labels_all[c]
    data_test.loc[(data_test.Stim ==c), "Stims"]  = labels_all[c]
data_test=data_test.drop(data_test[data_test.Region=='WM'].index)
data_test=data_test.drop(data_test[data_test.RegionC=='WM'].index)
data_test = data_test.drop(columns=['Region', 'RegionC'])
G       = nx.from_pandas_edgelist(data_test, "Stims", "Recs",["LL", "d"])
edges   = nx.to_pandas_edgelist(G)
nodes   = pd.DataFrame({'ID': labels_all, 'Region': labels_region, 'H': labels_L})
nodes   = nodes.drop(nodes[nodes.Region=='WM'].index)
G       = make_graph(nodes, edges)

plt.figure(figsize=(15,15))
pos = nx.circular_layout(G) #nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
                        node_size = 500)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos,edge_color='r', arrows=True)

plt.show()

In [ ]:
plt.figure(figsize=(15,15))
pos = nx.circular_layout(G) #nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
                        node_size = 500)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos,edge_color='r', arrows=True)

plt.show()

In [ ]:
plt.figure(figsize=(15,15))
nx.draw_networkx_edges(G, pos, arrows=True)
nx.draw_networkx_edge_labels(G,pos,edge_labels=nx.get_edge_attributes(G, 'LL'))#edge_labels=nx.get_edge_attributes(G, 'LL')
nx.draw(G,pos, node_size=100,edge_cmap=plt.cm.Reds)
plt.show()


In [ ]:
c = MatrixPlot(G,node_grouping='Region', node_color='H', edge_width=None, edge_color=None)
c.draw()

In [ ]:
G.nodes

In [ ]:
c = CircosPlot(G, node_color='Region', group_label_position="middle", group_label_offset=12, node_grouping='H', node_labels=True, node_label_layout='rotation', figsize=(8,8))

c.draw()

In [ ]:
c = CircosPlot(G, node_color='H', group_label_position="middle", group_label_offset=12, edge_width=(edges['LL'] / edges['LL'].quantile(0.8)).tolist(), node_grouping='Region', node_labels=True, node_label_layout='rotation', figsize=(8,8))

c.draw()

## LL on mean CCEP

In [ ]:
stimlist = stimlist.fillna(0)
hours    = np.unique(stimlist.h)


exp = 1
## calcualte mean CCEp and then take LL 
data_CCEP = np.zeros((1,8))
CCEP_mean = np.zeros((len(StimChans),len(labels_all),len(hours), 2000,2))
#LL_CCEP   = np.zeros(((len(StimChans),len(labels_all),len(hours),2)))
w = 0.25
for h in range(len(hours)):
    hs = hours[h]
    print(hs)
    for sc in range(len(StimChans)):
        ChanP  = StimChanNums[sc]
        s      = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 
        stim_spec                = stimlist[(stimlist.h ==hs)&(stimlist.ChanP ==ChanP)]#&(stimlist.noise ==0)
        stimNum                  = stim_spec.StimNum.values#[:,0]
        CCEP_mean[sc, :, h,:,0]  = np.nanmean(ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), axis=1)
        CCEP_mean[sc, :, h,:,1]  = np.nanstd(ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), axis=1)
        LL_all     = LL_funcs.get_LL_both(data=np.expand_dims(np.nanmean(ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), axis=1),1), Fs=Fs, IPI=np.array([0]), t_0=1, win=w)
        
        #LL_CCEP[sc, :, h, 0] = 
        
        val         = np.zeros((LL_all.shape[0], 8))
        val[:, 0]   = np.arange(LL_all.shape[0])                                         # response channel
        val[:, 1]   = ChanP
        val[:, 2]   = LL_all[:,0,1]
        val[:, 3]   = LL_all[:,0,1]
        val[:, 4]   = LL_all[:,0,1]
        val[:, 5]   = np.sqrt(((coord_all[s,0]-coord_all[:,0])**2)+((coord_all[s,1]-coord_all[:,1])**2)+((coord_all[s,2]-coord_all[:,2])**2))
        val[:, 6]   = hs
        data_CCEP    = np.concatenate((data_CCEP, val), axis=0)

data_CCEP = data_CCEP[1:-1, :] # remove first row (dummy row)


LL_CCEP = pd.DataFrame(
    {"Chan": data_CCEP[:, 0], "Stim": data_CCEP[:, 1], "LL": data_CCEP[:, 2],"Distance": data_CCEP[:, 5],"LL norm": data_CCEP[:, 4],"Hour": data_CCEP[:, 6]})
LL_CCEP.insert(0, 'Dist_cat', 0)
LL_CCEP.loc[(LL_CCEP.Distance < 15), 'Dist_cat'] = 1
LL_CCEP.loc[(LL_CCEP.Distance > 15)&(LL_CCEP.Distance < 25), 'Dist_cat'] = 2
LL_CCEP.loc[(LL_CCEP.Distance > 25)&(LL_CCEP.Distance < 40), 'Dist_cat'] = 3
LL_CCEP.loc[(LL_CCEP.Distance > 40)&(LL_CCEP.Distance < 60), 'Dist_cat'] = 4
LL_CCEP.loc[(LL_CCEP.Distance > 60), 'Dist_cat'] = 5
LL_CCEP.to_csv(path_patient + '/Analysis/BrainMapping/LL/LL_CCEP_'+str(exp)+'_'+str(w)+'s.csv', index=False,
              header=True)  # scat_plot = scat_plot.fillna(method='ffill')
print('Data saved')

In [ ]:
exp = 2
## calcualte mean CCEp and then take LL 
data_CCEP = np.zeros((1,8))
CCEP_mean = np.zeros((len(StimChans),len(labels_all),len(hours), 2000,2))
#LL_CCEP   = np.zeros(((len(StimChans),len(labels_all),len(hours),2)))
w = 0.25
for h in range(len(hours)):
    hs = hours[h]
    print(hs)
    for sc in range(len(StimChans)):
        ChanP  = StimChanNums[sc]
        s      = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 
        stim_spec                = stimlist[(stimlist.h ==hs)&(stimlist.ChanP ==ChanP)]#&(stimlist.noise ==0)
        stimNum                  = stim_spec.StimNum.values#[:,0]
        CCEP_mean[sc, :, h,:,0]  = np.nanmean(ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), axis=1)
        CCEP_mean[sc, :, h,:,1]  = np.nanstd(ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), axis=1)
        LL_all     = LL_funcs.get_LL_both(data=np.expand_dims(np.nanmean(ff.lp_filter(EEG_resp[:,stimNum,:],45,Fs), axis=1),1), Fs=Fs, IPI=np.array([0]), t_0=1, win=w)
        
        #LL_CCEP[sc, :, h, 0] = 
        
        val         = np.zeros((LL_all.shape[0], 8))
        val[:, 0]   = np.arange(LL_all.shape[0])                                         # response channel
        val[:, 1]   = ChanP
        val[:, 2]   = LL_all[:,0,1]
        val[:, 3]   = LL_all[:,0,1]
        val[:, 4]   = LL_all[:,0,1]
        val[:, 5]   = np.sqrt(((coord_all[s,0]-coord_all[:,0])**2)+((coord_all[s,1]-coord_all[:,1])**2)+((coord_all[s,2]-coord_all[:,2])**2))
        val[:, 6]   = hs
        data_CCEP    = np.concatenate((data_CCEP, val), axis=0)

data_CCEP = data_CCEP[1:-1, :] # remove first row (dummy row)


LL_CCEP = pd.DataFrame(
    {"Chan": data_CCEP[:, 0], "Stim": data_CCEP[:, 1], "LL": data_CCEP[:, 2],"Distance": data_CCEP[:, 5],"LL norm": data_CCEP[:, 4],"Hour": data_CCEP[:, 6]})
LL_CCEP.insert(0, 'Dist_cat', 0)
LL_CCEP.loc[(LL_CCEP.Distance < 15), 'Dist_cat'] = 1
LL_CCEP.loc[(LL_CCEP.Distance > 15)&(LL_CCEP.Distance < 25), 'Dist_cat'] = 2
LL_CCEP.loc[(LL_CCEP.Distance > 25)&(LL_CCEP.Distance < 40), 'Dist_cat'] = 3
LL_CCEP.loc[(LL_CCEP.Distance > 40)&(LL_CCEP.Distance < 60), 'Dist_cat'] = 4
LL_CCEP.loc[(LL_CCEP.Distance > 60), 'Dist_cat'] = 5
LL_CCEP.to_csv(path_patient + '/Analysis/BrainMapping/LL/LL_CCEP_'+str(exp)+'_'+str(w)+'s.csv', index=False,
              header=True)  # scat_plot = scat_plot.fillna(method='ffill')
print('Data saved')

adding Z- Score

In [ ]:
n_it = 200
stim_spec                = stimlist[(stimlist.noise ==0)]
stimNum                  = stim_spec.StimNum.values#[:,0]
LL_BL = np.zeros((len(labels_all),n_it,2))
LL_BL_z = np.zeros((len(labels_all),2))
for i in range(n_it):
    ix_choice = np.random.choice(len(stimNum), 3)
    LL                    = LL_funcs.get_LL_both(data=np.expand_dims(np.nanmean(ff.lp_filter(EEG_resp[:,stimNum[ix_choice],:],45,Fs), axis=1),1), Fs=Fs, IPI=np.array([0]), t_0=0.5, win=w)
    LL_BL[:,i,0] = LL[:,0,1] 
LL_BL_z[:,0] = np.nanmean(LL_BL[:,:,0],1)
LL_BL_z[:,1] = np.nanstd(LL_BL[:,:,0],1)
for c in range(len(labels_all)):
    LL_BL[c,:,1] = (LL_BL[c,:,0] - LL_BL_z[c,0])/LL_BL_z[c,1]

In [ ]:
for rc in range(len(labels_all)):
    LL_CCEP.loc[(LL_CCEP.Chan ==rc), 'LL norm'] = (LL_CCEP.loc[(LL_CCEP.Chan ==rc), 'LL']- LL_BL_z[rc,0])/LL_BL_z[rc,1]
    
LL_CCEP.loc[(LL_CCEP.Distance <7), 'LL'] = 0
LL_CCEP.loc[(LL_CCEP.Distance <7), 'LL norm'] = 0
LL_CCEP.loc[(LL_CCEP.Chan.isin(bad_chans)), 'LL'] = 0
LL_CCEP.loc[(LL_CCEP.Chan.isin(bad_chans)), 'LL norm'] = 0

In [ ]:

ChanP = StimChanNums[sc]
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(LL_BL[0,:,0], 50, alpha=0.75)
bincenters       = 0.5*(bins[1:]+bins[:-1])
y                = norm.pdf(bincenters, np.mean(LL_BL[0,:,0]), np.std(LL_BL[0,:,0]))
plt.plot(bincenters, y*30, 'r--', linewidth=1)
n, bins, patches = plt.hist(LL_CCEP.loc[(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Chan ==rc), 'LL'].values, alpha=0.75)
plt.title('Stim: ' + StimChans[sc] + ', Resp: ' + labels_all[rc], fontsize=16)
plt.xlabel('LL [uV/ms] of 250ms')

In [ ]:
plt.figure(figsize=(10,6))
ChanP = StimChanNums[sc]
n, bins, patches = plt.hist(LL_BL[0,:,1], 50, alpha=0.75)
bincenters       = 0.5*(bins[1:]+bins[:-1])
y                = norm.pdf(bincenters, np.mean(LL_BL[0,:,1]), np.std(LL_BL[0,:,1]))
plt.plot(bincenters, y*30, 'r--', linewidth=1)
n, bins, patches = plt.hist(LL_CCEP.loc[(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Chan ==rc), 'LL norm'].values, alpha=0.75)
plt.title('Stim: ' + StimChans[sc] + ', Resp: ' + labels_all[rc], fontsize=16)
plt.xlabel('LL z-scored')

In [ ]:
LL_CCEP

In [ ]:
LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==s)&(LL_CCEP.Hour ==hs)]

In [ ]:
rc = 8
sc = 36

ChanP = StimChanNums[sc]
s      = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 

hours = np.unique(stimlist.h)
hours = np.array([0,6,12,18])


fig = plt.figure(figsize=(11, 8))
gs  = fig.add_gridspec(2,2)  # GridSpec(4,1, height_ratios=[1,2,1,2])
plt.suptitle('Stim: ' + StimChans[sc] + ', Resp: ' + labels_all[rc], fontsize=16)
for i in range(len(hours)):
    hs = hours[i]
    stim_spec           = stimlist[(stimlist.ChanP ==ChanP)&(stimlist.h ==hs)]
    stimNum                  = stim_spec.StimNum.values#[:,0]
    CCEP_mean = np.nanmean(ff.lp_filter(EEG_resp[rc,stimNum,:],20,Fs), axis=0)
    CCEP_std = np.nanstd(ff.lp_filter(EEG_resp[rc,stimNum,:],20,Fs), axis=0)
    if i>1:
        axs                 = fig.add_subplot(gs[1,i-2])
    else:
        axs                 = fig.add_subplot(gs[0,i])
    
    plt.plot(x_ax,CCEP_mean)
    plt.fill_between(x_ax, CCEP_mean-CCEP_std, CCEP_mean+CCEP_std, alpha=0.1)
    plt.ylabel('uV', fontsize=12)
    plt.axvline(0, c=[1, 0, 0], linewidth=1.5)
    plt.title(str(hs)+':00')
    plt.xlim([-0.2,0.5])
    plt.ylim([-800,400])
    if np.max(CCEP_mean) >0:
        plt.text(0.3, -300, 'LL: '+str(np.round(np.mean(LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==s)&(LL_CCEP.Hour ==hs)]['LL'].values),2)))
        #plt.text(0.3, -200, 'zLL: '+str(np.round(LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Hour ==hs)]['LL norm'].values[0],2)))
    #plt.ylim(limy)

make matrix with 0 and 1 if there is a response based on mean z-score across hours (>1.96)

In [ ]:
bad_chans.shape

In [ ]:
M_resp = np.zeros((len(StimChanNums), len(labels_all)))
for sc in range(len(StimChans)):
    ChanP  = StimChanNums[sc]
    s = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 
    for rc in range(len(labels_all)):
        z_mean = np.nanmedian(LL_CCEP.loc[(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Chan ==rc), 'LL norm'].values)
        if z_mean > 1.96:
            M_resp[sc,rc] =1
        if s == rc:
            M_resp[sc,rc] =-1
M_resp[:,bad_chans] = 0


In [ ]:
# Plot distance matrix.
cmap = ListedColormap(['r', 'k', 'w'])
fig = pylab.figure(figsize=(25,15))
axmatrix = fig.add_axes([0.4,0.1,0.6,0.6])

im = axmatrix.matshow(M_resp,cmap=cmap, aspect='auto', origin='lower')

plt.xticks(range(len(labels_all)), labels_all, rotation=90);
plt.yticks(range(len(StimChanNums)), StimChans);
plt.show()

In [ ]:
# check data where there is propagated response
resp_ix = np.array(np.where(M_resp==1))

n = 145
sc = resp_ix[0,n]
rc = resp_ix[1,n]

ChanP = StimChanNums[sc]
s      = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 
hours = np.unique(stimlist.h)
hours = np.array([0,6,12,18])


fig = plt.figure(figsize=(11, 8))
gs  = fig.add_gridspec(2,2)  # GridSpec(4,1, height_ratios=[1,2,1,2])
plt.suptitle('Stim: ' + StimChans[sc] + ', Resp: ' + labels_all[rc], fontsize=16)
for i in range(len(hours)):
    hs = hours[i]
    stim_spec           = stimlist[(stimlist.ChanP ==ChanP)&(stimlist.h ==hs)]
    stimNum                  = stim_spec.StimNum.values#[:,0]
    CCEP_mean = np.nanmean(ff.lp_filter(EEG_resp[rc,stimNum,:],45,Fs), axis=0)
    CCEP_std = np.nanstd(ff.lp_filter(EEG_resp[rc,stimNum,:],45,Fs), axis=0)
    if i>1:
        axs                 = fig.add_subplot(gs[1,i-2])
    else:
        axs                 = fig.add_subplot(gs[0,i])
    if i == 0:
        mx = np.min([np.max([np.max(abs(CCEP_mean))*1.5,300]),800])
    plt.plot(x_ax,CCEP_mean)
    plt.fill_between(x_ax, CCEP_mean-CCEP_std, CCEP_mean+CCEP_std, alpha=0.1)
    plt.ylabel('uV', fontsize=12)
    plt.axvline(0, c=[1, 0, 0], linewidth=1.5)
    plt.title(str(hs)+':00')
    plt.xlim([-0.2,0.5])
    plt.ylim([-mx,mx])
    if np.max(CCEP_mean) >0:
        plt.text(0.3, -mx/2-100, 'LL: '+str(np.round(LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Hour ==hs)]['LL'].values[0],2)))
        plt.text(0.3, -mx/2, 'zLL: '+str(np.round(LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Hour ==hs)]['LL norm'].values[0],2)))
    #plt.ylim(limy)

## CR effect on brain map

In [ ]:
hours = np.unique(stimlist.h)

In [ ]:
M_hour = np.zeros((len(StimChanNums), len(labels_all), len(hours)))
for hs in range(len(hours)):
    h = hours[hs]
    for sc in range(len(StimChans)):
        ChanP  = StimChanNums[sc]
        s = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 
        if LL_CCEP.loc[(LL_CCEP.Hour ==h)&(LL_CCEP.Stim ==ChanP), 'LL norm'].values.shape[0]==110:
            M_hour[sc,:,hs] = LL_CCEP.loc[(LL_CCEP.Hour ==h)&(LL_CCEP.Stim ==ChanP), 'LL norm'].values 

In [ ]:
path_patient

In [ ]:
for h in range(len(hours)):
    fig = pylab.figure(figsize=(25,15))
    axmatrix = fig.add_axes([0.4,0.1,0.6,0.6])

    im = axmatrix.matshow(M_hour[:,:,h], aspect='auto', origin='lower', cmap='hot', vmin=1.96, vmax=10)

    plt.xticks(range(len(labels_all)), labels_all, rotation=90)
    plt.yticks(range(len(StimChanNums)), StimChans)
    plt.xlabel(str(hours[h])+':00', fontsize=16)
    plt.savefig(path_patient + '/Analysis/BrainMapping/LL/figures/BM_'+str(hours[h])+'.jpg')

In [ ]:
sc = 36
rc = 8

ChanP = StimChanNums[sc]
s      = np.where(labels_all == StimChans[sc])[0][0]#i#np.int(StimChanNums[i]) 
hours = np.unique(stimlist.h)
hours = np.array([0,6,12,18])


fig = plt.figure(figsize=(11, 8))
gs  = fig.add_gridspec(2,2)  # GridSpec(4,1, height_ratios=[1,2,1,2])
plt.suptitle('Stim: ' + StimChans[sc] + ', Resp: ' + labels_all[rc], fontsize=16)
for i in range(len(hours)):
    hs = hours[i]
    stim_spec           = stimlist[(stimlist.ChanP ==ChanP)&(stimlist.h ==hs)]
    stimNum                  = stim_spec.StimNum.values#[:,0]
    
    stim_list_rec   = stimlist[(stimlist['StimNum'].isin((stimNum-1).clip(min=0)))]
    ReChanP         = SM2IX(stim_list_rec.ChanP.values,StimChanNums,np.array(StimChanIx))
    #ReChanN = SM2IX(stim_list_rec.ChanN.values,StimChanNums,np.array(StimChanIx))
    stimNum         = stimNum[((ReChanP!=rc)&(ReChanP+1!=rc)&(ReChanP-1!=rc))|(stim_list_rec.ISI_s.values>4)]
            
            
    CCEP_mean = np.nanmean(ff.lp_filter(EEG_resp[rc,stimNum,:],45,Fs), axis=0)
    CCEP_std = np.nanstd(ff.lp_filter(EEG_resp[rc,stimNum,:],45,Fs), axis=0)
    if i>1:
        axs                 = fig.add_subplot(gs[1,i-2])
    else:
        axs                 = fig.add_subplot(gs[0,i])
    if i == 0:
        mx = np.min([np.max([np.max(abs(CCEP_mean))*1.5,300]),800])
    plt.plot(x_ax,CCEP_mean)
    plt.fill_between(x_ax, CCEP_mean-CCEP_std, CCEP_mean+CCEP_std, alpha=0.1)
    plt.ylabel('uV', fontsize=12)
    plt.axvline(0, c=[1, 0, 0], linewidth=1.5)
    plt.title(str(hs)+':00')
    plt.xlim([-0.2,0.5])
    plt.ylim([-mx,mx])
    if np.max(CCEP_mean) >0:
        plt.text(0.3, -mx/2-100, 'LL: '+str(np.round(LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==s)&(LL_CCEP.Hour ==hs)]['LL'].values[0],2)))
        #plt.text(0.3, -mx/2, 'zLL: '+str(np.round(LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==ChanP)&(LL_CCEP.Hour ==hs)]['LL norm'].values[0],2)))
    #plt.ylim(limy)

In [ ]:
chan_nums     = np.arange(len(labels_all))
chan_nums_req = chan_nums[req]
# plot ratio of LL of mean CCEP vs distance 
#data = LL_CCEP[~(LL_CCEP['Chan'].isin(bad_chans))&(LL_CCEP['Chan'].isin(chan_nums_req))&(LL_CCEP['Distance']>3.5)&(LL_CCEP['LL BL norm']>1.5)]
data = LL_CCEP[(LL_CCEP['Chan']==rc)&(LL_CCEP.Stim ==ChanP)]
fig    = plt.figure(figsize=(10, 8))

sns.catplot(x="Hour", y="LL",data=data, kind="swarm", ci="sd",
              height=6,  # make the plot 5 units high
              aspect=1.5, legend_out=True)
plt.xlabel('hour', fontsize=16)
plt.ylabel('LL', fontsize=16)
plt.tick_params(axis="both", labelsize=8)
#plt.ylim([0,5])
#plt.axhline(1, c=[0,0,0])
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.suptitle('Stim: ' + StimChans[sc] + ', Resp: ' + labels_all[rc], fontsize=16)
#plt.savefig(path_patient + '/Analysis/InputOutput/LL/figures/IO_Ph_'+StimChan+ '-'+str(labels_all[rc])+'.jpg')
plt.show()